In [165]:
# import different packages
import pandas as pd
import bqplot
# new one
import numpy as np
import ipywidgets

In [166]:
buildings = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv",
                       na_values = {'Year Acquired': 0, 
                                     'Year Constructed': 0, 
                                     'Square Footage': 0})
# data transformation is to load a link here.
# Here I was using dropna="False", but it didn't get rid of the zero values.
# So I just look at the pd.read_csv syntax, and I think this way may be more useful.


In [167]:
buildings_pivot = pd.pivot_table(buildings, index=["Agency Name"], values=["Square Footage"],
                                 columns=["County"], aggfunc=np.mean)
# Here I used pivot table, because we need the numerical data for the table. The numerical data is the index value.
# Values are mean of total square footage 
buildings_pivot
# Preview the data table

Square Footage                \
County                                             Adams     Alexander   
Agency Name                                                              
Appellate Court / Fifth District                     NaN           NaN   
Appellate Court / Fourth District                    NaN           NaN   
Appellate Court / Second District                    NaN           NaN   
Appellate Court / Third District                     NaN           NaN   
Chicago State University                             NaN           NaN   
Department of Agriculture                            NaN           NaN   
Department of Central Management Services            NaN           NaN   
Department of Corrections                    8613.333333  16422.863636   
Department of Human Services                         NaN           NaN   
Department of Juvenile Justice                       NaN           NaN   
Department of Military Affairs               5971.000000           NaN   
Department of Natural Resources              1209.888889   2496.000000   
Department of Public Health                          NaN           NaN   
Department of Revenue                                NaN           NaN   
Department of State Police                           NaN           NaN   
Department of Transportation                 3677.250000   2337.444444   
Department of Veterans' Affairs             12541.538462           NaN   
Eastern Illinois University                          NaN           NaN   
Governor's Office                                    NaN           NaN   
Governors State University                           NaN           NaN   
Historic Preservation Agency                         NaN           NaN   
IL State Board of Education                          NaN           NaN   
Illinois Board of Higher Education                   NaN           NaN   
Illinois Community College Board                     NaN           NaN   
Illinois Courts                                      NaN           NaN   
Illinois Emergency Management Agency                 NaN           NaN   
Illinois Medical District Commission                 NaN           NaN   
Illinois State University                            NaN           NaN   
Northeastern Illinois University                     NaN           NaN   
Northern Illinois University                         NaN           NaN   
Office of the Attorney General                       NaN           NaN   
Office of the Secretary of State                     NaN           NaN   
Southern Illinois University                         NaN           NaN   
University of Illinois                               NaN           NaN   
Western Illinois University                          NaN           NaN   

                                                                          \
County                                        Bond   Boone         Brown   
Agency Name                                                                
Appellate Court / Fifth District               NaN     NaN           NaN   
Appellate Court / Fourth District              NaN     NaN           NaN   
Appellate Court / Second District              NaN     NaN           NaN   
Appellate Court / Third District               NaN     NaN           NaN   
Chicago State University                       NaN     NaN           NaN   
Department of Agriculture                      NaN     NaN           NaN   
Department of Central Management Services      NaN     NaN           NaN   
Department of Corrections                      NaN     NaN  16046.307692   
Department of Human Services                   NaN     NaN           NaN   
Department of Juvenile Justice                 NaN     NaN           NaN   
Department of Military Affairs                 NaN     NaN           NaN   
Department of Natural Resources                NaN     NaN           NaN   
Department of Public Health                    NaN     NaN           NaN   
Department of

In [168]:
# heatmap
# 1. Data as previous

# 2. Scales -- how will data translate to Marks
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
# A mapping from a discrete set of values to a numerical range. So we will need a ordinal scale here.

col_sc = bqplot.ColorScale(scheme="Blue") 
# like x/y scale, but in "color space" Since the background color is black, so here I choose a light color.

# 3. Axis
ax_x = bqplot.Axis(scale=x_sc,label='County', label_offset='100px',
                  tick_rotate=90)
# rotate the label 90 degress, mapping between data and the range
ax_y = bqplot.Axis(scale=y_sc, label="Y Value", orientation="vertical")
# vertical label, scale used y_sc
ax_col = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right') # color bars
# here is the color scale, scale also used previous scale

# 4. Marks -- how we draw the data
anames = buildings_pivot.columns.levels[1].to_list()
# levels[1] means the second level in the coulumn, and we covert it into a list.
heat_map = bqplot.GridHeatMap(color = np.log10(buildings_pivot.values),# return the base 10 logarithm of the array
                              row = buildings_pivot.index, # congressinal district
                              column = anames,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'}, # use interaction feature
                              anchor_style = {'fill':'blue'}) 
    
# heat_map = bqplot.GridHeatMap(color=buildings_pivot,
#                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc},
#                               anchor_style={'fill':'blue'},interactions={'click':'select'})
 
# 5. Figure object
fig = bqplot.Figure(marks=[heat_map], axes=[ax_col,ax_x,ax_y])
# myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
# myDashboard
fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(scheme='Blue'), side='right'), Axis(label='Cou…

In [178]:
x_scl = bqplot.DateScale() # demonstrate the time
y_scl = bqplot.LinearScale() # let it rescale each time, for numerical data
ax_xcl = bqplot.Axis(label='Date', scale=x_scl) 
ax_ycl = bqplot.Axis(label='Square footage in year acquired', scale=y_scl)

# plot
bar = bqplot.Bars(x = years, y = sqft,
                    scales = {'x': x_scl, 'y': y_scl})

fig_bar = bqplot.Figure(marks=[bar], axes=[ax_xcl,ax_ycl])

i,j = 0,0
mask = (buildings['Agency Name'] == buildings_pivot.index[i]) & \
       (buildings['County'] == buildings_pivot.columns.levels[1][j]) 
# use index as rows, and I used level method again for the columns values
# new, subset dataframe
buildings_subset = buildings[mask]
# grab years and sqft from pandas
grouped = buildings_subset.groupby("Year Acquired")["Square Footage"].sum()
years = grouped.index
# x is the year
sqft = grouped.values
# y is total square footage acquired that year

# Bar
bar = bqplot.Bars(x = years, y = sqft,
                    scales = {'x': x_scl, 'y': y_scl}) # we need x,y,and scales to build a bar
fig_bar = bqplot.Figure(marks = [bar], axes = [ax_y, ax_x])

# interaction function
def get_data_value(change):
    i,j = change['owner'].selected[0] # the only selected bin
    mask = (buildings['Agency Name'] == buildings_pivot.index[i]) & \
    (buildings['County'] == buildings_pivot.columns.levels[1][j])
    # grab buildings subset dataframe
    buildings_subset = buildings[mask]
    # grab years and sqft from pandas
    grouped = buildings_subset.groupby("Year Acquired")["Square Footage"].sum()
    # Values are mean of total square footage for that dataset
    years = grouped.index
    sqft = grouped.values
    bar.x = years
    bar.y = sqft
    
# link interaction function to my mark
heat_map.observe(get_data_value, 'selected')

# there are some fancier layout things in here that they may/may not have and that's fine
fig = bqplot.Figure(marks = [heat_map], axes = [ax_col, ax_x, ax_y], 
                    fig_margin = dict(top=50, bottom=150, left=50, right=50))
# fig position

fig.layout.align_items
fig.layout.min_width='500px'
fig.layout.min_height='500px'
fig_bar.layout.min_width='500px'
# fig setting, subplot. I made it into half and half.

ipywidgets.HBox([fig,fig_bar])
# subplot